### MTH 459/559 - ch 16 binomial tree method

In [1]:
# ch16_binomial_tree.ipynb  
import numpy as np # library for numerical & math calculations
import matplotlib.pyplot as plt # library for graphing 
import scipy.stats as sp # library with prob/stat functions

In [2]:
# functions for exact value of European Call for comparison
def d1(S,K,T,t,r,sigma):
    d1 = (np.log(S/K) + (r+0.5*sigma**2)*(T-t))/(sigma*np.sqrt(T-t))
    return d1
def d2(S,K,T,t,r,sigma):
    d2 = (np.log(S/K) + (r-0.5*sigma**2)*(T-t))/(sigma*np.sqrt(T-t))
    return d2
def C_euro(S,K,T,t,r,sigma):
    C_euro = S*sp.norm.cdf(d1(S,K,T,t,r,sigma))-\
        K*np.exp(-r*(T-t))*sp.norm.cdf(d2(S,K,T,t,r,sigma)) 
    return C_euro

In [3]:
# binomial tree method of option valuation

# stock paramters
S0 = 9
K = 10
T = 3.
r = 0.06
sigma = 0.3

# binomial parameters for stock model
M = 100
dt = T/M
p = 0.5
nu = r - 0.5*sigma**2
u = np.exp(sigma*np.sqrt(dt)+nu*dt)
d = np.exp(-sigma*np.sqrt(dt)+nu*dt)
#print('p,u,d = ',p,u,d)

# stock model 
from scipy.special import comb as nchoosek
S = np.zeros(M+1) # array of S values at t=T
prob = np.zeros(M+1)  #probability at t=T
for n in range(M+1): # n = 0,1,...,M
    S[n]=S0*d**(M-n)*u**(n)   # M-n down steps and n up steps
    prob[n] = nchoosek(M,n)*p**n*(1-p)**(M-n)
#print('S_T    = ',S)
#print(prob,sum(prob))

# compare probability distribution of S_T to asset model
#Z = (np.log(S/S0) - nu*T)/(sigma*np.sqrt(T))
#binwidth = np.log(u/d)/(sigma*np.sqrt(T))
#plt.plot(Z,prob/binwidth,'bo')
#xfine = np.linspace(-3,3) # fine array in x for plotting exact curve
#plt.plot(xfine,sp.norm.pdf(xfine),'r-')
#plt.xlabel('z')
#plt.ylabel('probability')
#plt.show()

# binomial tree for option values
V = np.zeros((M+1, M+1))*np.NaN # V(i,j) = option value at t_i, n down steps

# payout values at t=T
for n in range(M+1): # n = 0,1,...,M
    V[M,n] = max(S[n]-K,0)    # payout function at t=T
#print(V[M,:])
    
# propagate option value backwards using (16.3)
for i in range(M-1, -1, -1): # i = M-1,M-2,...,0 
    for n in range(i+1):     # n = 0,1,...,i
        V[i,n]=np.exp(-r*dt)*(p*V[i+1,n+1]+(1-p)*V[i+1,n]) 
#print(V)
price = V[0,0]

# compare to exact option price
print('binomial option price      = ',price)
print('Black-Scholes option price = ',C_euro(S0,K,T,0,r,sigma))
print('error = ',price-C_euro(S0,K,T,0,r,sigma))

binomial option price      =  2.123308242012514
Black-Scholes option price =  2.1200938314108675
error =  0.0032144106016462892


Note - Error ->0 but not monotonic in M (see text sec 16.4):
* Error distinctly different for M even vs M odd.  
* error has decaying oscillations as M increases
* overall, error can be bounded by const/M  